In [ ]:
#this is where I will be merging all 3 of my data sets
import pandas as pd
import numpy as np"


In [ ]:
#data set 1
regular1 = pd.read_csv("regular1.csv")
regular2 = pd.read_csv("regular2.csv")
regular3 = pd.read_csv("regular3.csv")